In [1]:
import cv2 as cv
import numpy as np

In [2]:
hsv = 0
lower_blue1 = 0
upper_blue1 = 0
lower_blue2 = 0
upper_blue2 = 0
lower_blue3 = 0
upper_blue3 = 0
bins = np.arange(256).reshape(256,1)

In [3]:
def nothing(x):
    pass

def mouse_callback(event, x, y, flags, param):
    global hsv, lower_blue1, upper_blue1, lower_blue2, upper_blue2, lower_blue3, upper_blue3, threshold

    # 마우스 클릭 위치 픽셀값을 읽어와서 HSV로 변환
    if event == cv.EVENT_LBUTTONDOWN:
        print(img_color[y, x])
        color = img_color[y, x]
        
        
        one_pixel = np.uint8([[color]])
        #HSV 이미지로 변환
        #색상 : H ) 0 - 360, 가장 파장이 긴 빨간색을 0도로 지정한다.
        #채도 : S ) 0 - 100, 색상이 가장 진한 상태를 100으로 하며, 진함의 정도
        #명도 : V ) 0 - 100 , 흰색, 빨간색을 100, 검은색이 0이며, 밝은 정도
        hsv = cv.cvtColor(one_pixel, cv.COLOR_BGR2HSV)
        # 픽셀값을 가져옴
        hsv = hsv[0][0]

        threshold = cv.getTrackbarPos('threshold', 'img_result')
        
        # HSV 색공간에서 마우스 클릭으로 얻은 픽셀값과 유사한 필셀값의 범위를 정합니다.
        if hsv[0] < 10:
            print("case1")
            lower_blue1 = np.array([hsv[0]-10+180, threshold, threshold])
            upper_blue1 = np.array([180, 255, 255])
            lower_blue2 = np.array([0, threshold, threshold])
            upper_blue2 = np.array([hsv[0], 255, 255])
            lower_blue3 = np.array([hsv[0], threshold, threshold])
            upper_blue3 = np.array([hsv[0]+10, 255, 255])

        elif hsv[0] > 170:
            print("case2")
            lower_blue1 = np.array([hsv[0], threshold, threshold])
            upper_blue1 = np.array([180, 255, 255])
            lower_blue2 = np.array([0, threshold, threshold])
            upper_blue2 = np.array([hsv[0]+10-180, 255, 255])
            lower_blue3 = np.array([hsv[0]-10, threshold, threshold])
            upper_blue3 = np.array([hsv[0], 255, 255])

            
        else:
            print("case3")
            lower_blue1 = np.array([hsv[0], threshold, threshold])
            upper_blue1 = np.array([hsv[0]+10, 255, 255])
            lower_blue2 = np.array([hsv[0]-10, threshold, threshold])
            upper_blue2 = np.array([hsv[0], 255, 255])
            lower_blue3 = np.array([hsv[0]-10, threshold, threshold])
            upper_blue3 = np.array([hsv[0], 255, 255])


        print(hsv[0])
        print("@1", lower_blue1, "~", upper_blue1)
        print("@2", lower_blue2, "~", upper_blue2)
        print("@3", lower_blue3, "~", upper_blue3)
        
def draw_histogram(img):

    h = np.zeros((img.shape[0], 513), dtype=np.uint8)

    hist_item = cv.calcHist([img],[0],None,[256],[0,256])
    cv.normalize(hist_item,hist_item,0,255,cv.NORM_MINMAX)
    hist=np.int32(np.around(hist_item))
    for x,y in enumerate(hist):
        cv.line(h,(x,0+10),(x,y+10),(255,255,255))

    cv.line(h, (0, 0 + 10), (0, 5), (255, 255, 255) )
    cv.line(h, (255, 0 + 10), (255, 5), (255, 255, 255))
    y = np.flipud(h)

    #draw curve
    hist, bin = np.histogram(img.flatten(), 256, [0, 256])
    cdf = hist.cumsum()
    cdf_normalized = cdf * float(hist.max()) / cdf.max()
    cv.normalize(cdf_normalized, cdf_normalized, 0, 255, cv.NORM_MINMAX)
    hist = np.int32(np.around(cdf_normalized))
    pts = np.int32(np.column_stack((bins, hist)))
    pts += [257, 10]

    cv.line(h, (0+257, 0 + 10), (0+257, 5), (255, 255, 255) )
    cv.line(h, (255+257, 0 + 10), (255+257, 5), (255, 255, 255))
    cv.polylines(h, [pts], False, (255,255,255))

    return y

In [5]:
cv.namedWindow('img_color')
cv.setMouseCallback('img_color', mouse_callback)

cv.namedWindow('img_result')
cv.createTrackbar('threshold','img_result')
cv.setTrackbarPos('threshold', 'img_result', 30)

#cap0 = cv.VideoCapture(0)

TypeError: function takes exactly 5 arguments (2 given)

In [37]:
while(True):
    img_color = cv.imread('cam2_720p.jpg')
    #ret,img_color = cap.read()
    height, width = img_color.shape[:2]
    img_color = cv.resize(img_color, (width, height), interpolation=cv.INTER_AREA)

    # 원본 영상을 HSV 영상으로 변환합니다.
    img_hsv = cv.cvtColor(img_color, cv.COLOR_BGR2HSV)

    # 범위 값으로 HSV 이미지에서 마스크를 생성합니다.
    img_mask1 = cv.inRange(img_hsv, lower_blue1, upper_blue1)
    img_mask2 = cv.inRange(img_hsv, lower_blue2, upper_blue2)
    img_mask3 = cv.inRange(img_hsv, lower_blue3, upper_blue3)
    img_mask = img_mask1 | img_mask2 | img_mask3

    kernel = np.ones((11,11), np.uint8)
    img_mask = cv.morphologyEx(img_mask, cv.MORPH_OPEN, kernel)
    img_mask = cv.morphologyEx(img_mask, cv.MORPH_CLOSE, kernel)

    # 마스크 이미지로 원본 이미지에서 범위값에 해당되는 영상 부분을 획득합니다.
    img_result = cv.bitwise_and(img_color, img_color, mask=img_mask)


    numOfLabels, img_label, stats, centroids = cv.connectedComponentsWithStats(img_mask)

    for idx, centroid in enumerate(centroids):
        if stats[idx][0] == 0 and stats[idx][1] == 0:
            continue

        if np.any(np.isnan(centroid)):
            continue

        x,y,width,height,area = stats[idx]
        centerX,centerY = int(centroid[0]), int(centroid[1])
        print(centerX, centerY)

        if area > 50:
            cv.circle(img_color, (centerX, centerY), 10, (0,0,255), 10)
            cv.rectangle(img_color, (x,y), (x+width,y+height), (0,0,255))

    cv.imshow('img_color', img_color)
    cv.imshow('img_mask', img_mask)
    cv.imshow('img_result', img_result)


    # ESC 키누르면 종료
    if cv.waitKey(1) & 0xFF == 27:
        break


cv.destroyAllWindows()

1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1

error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-c2l3r8zm\opencv\modules\highgui\src\window.cpp:868: error: (-215:Assertion failed) trackbar in function 'cv::getTrackbarPos'


1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
[ 55 182 250]


error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-c2l3r8zm\opencv\modules\highgui\src\window.cpp:868: error: (-215:Assertion failed) trackbar in function 'cv::getTrackbarPos'


1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
[ 48 166 255]


error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-c2l3r8zm\opencv\modules\highgui\src\window.cpp:868: error: (-215:Assertion failed) trackbar in function 'cv::getTrackbarPos'


1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
[ 48 166 255]


error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-c2l3r8zm\opencv\modules\highgui\src\window.cpp:868: error: (-215:Assertion failed) trackbar in function 'cv::getTrackbarPos'


1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
1272 161
